In [328]:
import numpy as np
import itertools
import math

In [ ]:
N = 2

In [175]:
s0 = np.matrix([[1,0],[0,1]])
s1 = np.matrix([[0,1],[1,0]])
s2 = np.matrix([[0,-1j],[1j,0]])
s3 = np.matrix([[1,0],[0,-1]])
s = np.array((s0,s1,s2,s3))
s.shape

(4, 2, 2)

In [324]:
def random_herm(dim=2):
    m = np.random.rand(dim,dim) + 1j*np.random.rand(dim,dim)
    h = m + m.T.conj()
    return h
def random_matrix(dim=2):
    m = np.random.rand(dim,dim) + 1j*np.random.rand(dim,dim)
    return m

In [292]:
def make_basis_element(J):
    n = len(J)
    sigma = 1.
    for k in range(n):
        sigma = np.kron(sigma, s[J[k]])
    return sigma

In [300]:
def make_all_indices(n):
    base = (0,1,2,3)
    J = []
    for j in itertools.product(base,repeat=n):
        J.append(j)
    return J

In [320]:
def get_coefficient(j, M):
    sigma_j = make_basis_element(j)
    cj = np.einsum("ij,ji",sigma_j, M)
    return cj

In [366]:
def coefficients_to_matrix(c):
    n = int(math.log(len(c),4))
    dim = 2**n
    M = np.zeros((dim,dim),dtype=complex)

    for i, c_i in c.items():
        sigma = make_basis_element(i)
        M += c_i*sigma

    M = M/dim
    return M

def matrix_to_coefficients(M):
    dim = M.shape[0]
    n = int(math.log(dim,2))
    J = make_all_indices(n)
    c = {}
    for j in J:
        c[j] = get_coefficient(j,M)
    return c

In [355]:
n = 4
J = make_all_indices(n)
K = make_all_indices(n)

for j, k in itertools.product(J,K):
    sigma_j = make_basis_element(j)
    sigma_k = make_basis_element(k)
    trace = np.einsum("ij,ji",sigma_j,sigma_k)
    if j==k:
        assert np.isclose(trace, 2**n)
    else:
        assert trace == 0


In [367]:
n = 7
M = random_herm(2**n)
M_new = coefficients_to_matrix(matrix_to_coefficients(M))
assert np.allclose(M_new,M)

In [371]:
n = 3
M = random_herm(2**n)
matrix_to_coefficients(M)

{(0, 0, 0): (6.8748159271180675+0j),
 (0, 0, 1): (10.059720805231054+0j),
 (0, 0, 2): (-0.032620877176601226+0j),
 (0, 0, 3): (-1.3073557822825697+0j),
 (0, 1, 0): (6.4858893549845265+0j),
 (0, 1, 1): (8.919639352229405-1.1102230246251565e-16j),
 (0, 1, 2): (-0.633910823912985+0j),
 (0, 1, 3): (-0.5016501384944856-2.220446049250313e-16j),
 (0, 2, 0): (-1.5494112119611998+2.220446049250313e-16j),
 (0, 2, 1): (-0.11298250184778236+4.440892098500626e-16j),
 (0, 2, 2): (1.1287095199480972+0j),
 (0, 2, 3): (-4.236862408613096+0j),
 (0, 3, 0): (-1.0276877763049135+0j),
 (0, 3, 1): (2.126272117179325+0j),
 (0, 3, 2): (-1.216233207105447+0j),
 (0, 3, 3): (-1.0187994874030732+0j),
 (1, 0, 0): (8.771045902464909+0j),
 (1, 0, 1): (6.9845404066123+2.220446049250313e-16j),
 (1, 0, 2): (0.7508717619658936+0j),
 (1, 0, 3): (1.494902105788466+0j),
 (1, 1, 0): (4.215370742058774+0j),
 (1, 1, 1): (9.135615344707556+0j),
 (1, 1, 2): (-0.3438457036831317+0j),
 (1, 1, 3): (2.2667136383144313+0j),
 (1, 2, 0

In [331]:
def matrix_to_bloch(m):
    r0 = m[0,0] + m[1,1]
    r1 = (m[1,0] + m[0,1])
    r2 = (1j*(m[0,1]-m[1,0]))
    r3 = (m[0,0] - m[1,1])
    r = 1/2*np.array((r0,r1,r2,r3))
    return r

def bloch_to_matrix(r):
    #m = 1/2*np.einsum("i,ijk->jk",r,s)
    #m = 1/2*(r[0]*s0+r[1]*s1+r[2]*s2+r[3]*s3)
    #normalization = np.sqrt(2*(1+np.linalg.norm(r)**2))
    m =     np.array([
                    [r[0]+r[3],r[1]-1j*r[2]],
                    [r[1]+1j*r[2],r[0]-r[3]]
                    ])
    return m